In [2]:
import pandas as pd 

df = pd.read_csv("data_6_3_1.csv")
print(df['항암제'].values.tolist())

['무증상', '속쓰림', '무증상', '무증상', '조금아픔', '무증상', '조금아픔', '무증상', '조금아픔', '무증상', '아픔', '무증상', '조금아픔', '무증상', '아픔', '무증상', '속쓰림', '무증상', '아픔', '무증상']


### 문제 1. 항암제 투여 환자들 중 '무증상' 비율을 0과 1사이로 구하시오. 

In [3]:
df['항암제'].value_counts(normalize=True)

항암제
무증상     0.55
조금아픔    0.20
아픔      0.15
속쓰림     0.10
Name: proportion, dtype: float64

In [4]:
result = df['항암제'].value_counts(normalize=True)[0]
print(result)

0.55


/var/folders/kt/9_rb87_95nl_6glc6tz14j9m0000gn/T/ipykernel_63302/1578051067.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  result = df['항암제'].value_counts(normalize=True)[0]


In [5]:
cnt = df['항암제'] == '무증상'
total = len(df['항암제'])
result = sum(cnt) / total
print(result)

0.55


### 문제 2. 검정통계량
- 감기약의 부작용 비율과 항암제의 관찰 값이 통계적으로 유의미하게 차이가 있는지 카이제곱 검정을 이용하여 검정통계량을 구하시오. (적합도 검정)
- 가설검정
    + 귀무가설 : 감기약의 부작용과 항암제의 부작용은 동일하다.
    + 대립가설 : 감기약의 부작용과 항암제의 부작용은 다르다. 

In [6]:
df['항암제'].value_counts()

항암제
무증상     11
조금아픔     4
아픔       3
속쓰림      2
Name: count, dtype: int64

In [7]:
total = len(df['항암제'])
ratios = [0.7, 0.1, 0.05, 0.15] # 무증상, 조금아픔, 아픔, 속쓰림
val1 = df['항암제'].value_counts().to_list() # 무증상, 조금아픔, 아픔, 속쓰림
val2 = [ratios[0] * total, ratios[1] * total, ratios[2] * total, ratios[3] * total]
print(val1, val2) # 두 비율 비교

[11, 4, 3, 2] [14.0, 2.0, 1.0, 3.0]


In [8]:
import scipy.stats as stats
statistic, pvalue = stats.chisquare(val1, val2)
print(statistic)

6.976190476190475


### 문제 3. P-value 구하기
- p-value 값 구하기

In [9]:
print(pvalue)

0.07266054733847582


## 유형 2
- 주어진 데이터를 이용하여 다중선형회귀 문제를 해결하시오. 

In [11]:
import pandas as pd 

df = pd.read_csv("data_6_3_2.csv")
df.head()

,Ozone,Solar,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
4,NaN,NaN,14.3,56,5,5


### 문제 1. 
- 선형회귀 모델을 만들어 독립변수 Ozone의 회귀계수(Regression coefficient)를 구하시오.
    + 독립변수 : Solar(태양 에너지), Wind(바람의 세기), Ozone(오존 농도)
    + 종속변수 : Temp(온도)

In [18]:
from statsmodels.formula.api import ols

formula = "Temp ~ Solar + Wind + Ozone"
model = ols(formula, data=df).fit()
print(model.params)
print("---")
# 구하고자 하는 것이 오존의 회귀식
result = model.params['Ozone']
print(result)

Intercept    72.418579
Solar         0.007276
Wind         -0.322945
Ozone         0.171966
dtype: float64
---
0.17196604197981563


In [19]:
# 모델의 요약정보 출력
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   Temp   R-squared:                       0.500
Model:                            OLS   Adj. R-squared:                  0.486
Method:                 Least Squares   F-statistic:                     35.65
Date:                Fri, 12 Jan 2024   Prob (F-statistic):           4.73e-16
Time:                        15:11:24   Log-Likelihood:                -368.79
No. Observations:                 111   AIC:                             745.6
Df Residuals:                     107   BIC:                             756.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     72.4186      3.216     22.522      0.0

### 문제 2. 
- Solar와 Ozone은 고정, Wind가 증가함에 따라 Temp가 감소하는지 검증하기 위한 다중선형회귀 분석을 수행하여 Wind의 회귀계수에 대한 p-value를 구하시오. (유의수준 0.05)

In [15]:
print(model.pvalues)
result = model.pvalues['Wind']
print(result)

Intercept    2.107000e-42
Solar        3.454492e-01
Wind         1.690987e-01
Ozone        2.423506e-09
dtype: float64
0.16909868598497724


### 문제 3. 
- Solar가 150, Wind가 7.5, Ozone이 45일 경우, 예측 값과 그에 대한 95% 신뢰구간을 구하시오. 

In [17]:
new_data = pd.DataFrame({'Ozone':[45], 'Solar':[150], 'Wind':[7.5]})

prediction = model.get_prediction(new_data)
# 95% 신뢰구간 구하기
summary = prediction.summary_frame(alpha=0.05)
print(summary)

        mean   mean_se  mean_ci_lower  mean_ci_upper  obs_ci_lower  \
0  78.826312  0.864464      77.112614      80.540011     65.171661   

   obs_ci_upper  
0     92.480964  


In [65]:
print(summary[['mean_ci_lower', 'mean_ci_upper']])

   mean_ci_lower  mean_ci_upper
0      77.112614      80.540011


In [32]:
lower_bound = prediction.conf_int()[0][0]
upper_bound = prediction.conf_int()[0][1]

print(f"Confidence Interval (95%): [{lower_bound}, {upper_bound}]")

Confidence Interval (95%): [77.11261368775156, 80.54001091680799]


## tips 데이터 활용해서 회귀분석 진행
- 종속변수 : tip
- 독립변수 : 자유

In [22]:
import seaborn as sns
tips = sns.load_dataset('tips')
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


### 문제 1.¶
- 선형회귀 모델을 만들어 독립변수 total_bill의 회귀계수(Regression coefficient)를 구하시오.
  + 독립변수 : total_bill, sex, day
  + 종속변수 : tip

In [35]:
from statsmodels.formula.api import ols

formula = "tip ~ total_bill + sex + day"
model = ols(formula, data=tips).fit()
print(model.params)
print("---")
# 구하고자 하는 것이 total_bill의 회귀식
result = model.params['total_bill']
print(result)

Intercept        0.896801
sex[T.Female]    0.038442
day[T.Fri]       0.020633
day[T.Sat]      -0.060243
day[T.Sun]       0.103450
total_bill       0.104894
dtype: float64
---
0.10489376866174313


### 문제 1.¶
- 선형회귀 모델을 만들어 독립변수 day의 회귀계수(Regression coefficient)를 구하시오.
  + 독립변수 : total_bill, sex, day
  + 종속변수 : tip

In [36]:
from statsmodels.formula.api import ols

formula = "tip ~ total_bill + sex + day"
model = ols(formula, data=tips).fit()
print(model.params)
print("---")
# 구하고자 하는 것이 day의 회귀식
result = model.params['day']
print(result)

Intercept        0.896801
sex[T.Female]    0.038442
day[T.Fri]       0.020633
day[T.Sat]      -0.060243
day[T.Sun]       0.103450
total_bill       0.104894
dtype: float64
---


KeyError: 'day'

### 문제 2. 
- sex와 day는 고정, total_bil이 증가함에 따라 tip이 감소하는지 검증하기 위한 다중선형회귀 분석을 수행하여 total_bil의 회귀계수에 대한 p-value를 구하시오. (유의수준 0.05)

In [31]:
print(model.pvalues)
result = model.pvalues['total_bill']
print(result)

Intercept        2.031670e-05
sex[T.Female]    7.870593e-01
day[T.Fri]       9.390954e-01
day[T.Sat]       7.295158e-01
day[T.Sun]       5.701589e-01
total_bill       2.417821e-32
dtype: float64
2.4178208790145567e-32


### 문제 3. 
- Solar가 150, Wind가 7.5, Ozone이 45일 경우, 예측 값과 그에 대한 95% 신뢰구간을 구하시오. 